## Краулер для Indicator.ru ##

Импортируем все необходимые методы для последующей работы

In [1]:
import re
import urllib.request
import bs4 as bs
import pandas as pd
import os

Создаем пустой датафрейм. Он пригодится нам в дальнейшем для сортировки данных

In [40]:
ready_df = pd.DataFrame()

## Cкачиваем страницы ##
Функция для скачивания контента по ссылке в виде файла. Аргумент функции - ссылка. Результат работы кода функции - файл в директории "Pages".
Уникальное имя файла генерируется за счет части url (на основе анализа ссылок на статьи в Indicator.ru) за счет замены / на _ и отсекания общей части для всех url.

In [ ]:
def download(url):
    source = urllib.request.urlopen(url).read().decode('utf-8')
    file_name = url.split('https://indicator.ru/article/')[1].replace('/','_')
    fl = open('pages_indicator/' + file_name, 'w')
    fl.write(str(source))
    fl.close()

## Записываем тексты статей в отдельные файлы ##

Функция принимает в качестве аргумента текст и имя файла для записи (папка clear_text создана заранее). Данная функция понадобится нам в следующей функции.

In [3]:
def save_clear_text(text, file_name):
    f = open('clear_text_indicator/'+file_name + '.txt', 'w')
    f.write(text)
    f.close()

## Чистим текст, размечаем данные ##
    Функция принимает на вход путь к файлу (из ранее сгенерированной директории "Pages"). При помощи библиотеки Beautiful Soup на основании предварительного анализа структуры статей сайта находим элементы статьи: 
    - путь до текстового файла на Яндекс-диске
    - источник
    - url
    - дату (дату берем из ссылки на статью при помощи регулярных выражений)
    - заголовок
    - подзаголовок
    - автор (позаголовка и автора может не быть, поэтому в случае их отсутствия пишем None)
    - рубрика
    - тэги
    - жанр
    
    Полученные данные добавляем в заранее заготовленный пустой датафрейм; каждая колонка соответсвует вышеперечисленным элементам

In [41]:
def clear(file_path):
    global ready_df #связывает локальную переменную в коде функции с глобальной переменной в общем коде (задали в начале кода)
    with open(file_path) as fl:
        source = fl.read()
    
    soup_article = bs.BeautifulSoup(source, 'lxml')
    
    clear_p = ''    
    for paragraph in soup_article.find_all('div', class_ = 'article__body js-article-body'):
        clear_p = clear_p + paragraph.text
    
    head_title = soup_article.find('div', class_='headline__title').text

    rubric = soup_article.find('div', class_= 'headline__part').text

    date_raw = re.search(r'.*([0-9]{4}_[0-9]{2}_[0-9]{2}).*', file_path)
    date = date_raw.group(1)

    subtitle_raw = soup_article.find('div', class_ = 'headline__subtitle')
    if subtitle_raw == None:
        subtitle = 'None'
    else:
        subtitle = subtitle_raw.text
    
    author_raw = soup_article.find('div', class_ = 'article__author')
    if author_raw == None:
        author = ''
    else:
        author = author_raw.text.replace('Автор:','').strip()
        
    genre = 'Статьи'
    tags = ''
    path = file_path.replace('./pages_indicator', '/indicator_clear_text')
    site = 'indicator.ru'
    f_name = file_path.replace('pages_indicator/','')
    name = f_name.replace('./', '')
    url = 'https://indicator.ru/article/' + name.replace('_','/')
    
    
    #print('Path: %s, name: %s'%(path, name))
    df = pd.DataFrame([[path, site, url, date, head_title, subtitle, author, rubric, tags, genre]],
                      columns=['path', 'source', 'url', 'date','title','subtitle','author','rubrics','tags', 'genre'])
    ready_df = ready_df.append(df)
    save_clear_text(clear_p, name)
    

## Код ##

   К адресу сайта indicator.ru добавляем robots.txt => находим https://indicator.ru/sitemap.xml => находим все ссылки на статьи https://indicator.ru/sitemap-articles.xml.
   При помощи Beuatiful Soup выделяем адреса ссылок.
   Циклом при помощи написанной выше функции скачивания проходим по всем ссылкам статей и скачиваем их в папку "Pages".

In [52]:
sitemap_articles = urllib.request.urlopen('https://indicator.ru/sitemap-articles.xml').read()
soup_links = bs.BeautifulSoup(sitemap_articles, 'lxml')

for article_url in  soup_links.find_all('loc'):
    download(article_url.text)


Проходим по скачанным файлам в папке "Pages_indicator", отсеиваем служебные, остальные файлы чистим ранее написанной функцией clear.

In [42]:
start_path = './pages_indicator/'

for path, dirs, files in os.walk(start_path):

    for fname in files:
        if not fname.startswith('.'):
            clear(start_path+fname)

In [48]:
ready_df.head()

,path,source,url,date,title,subtitle,author,rubrics,tags,genre
0,/pages_indicator/2017_01_04_chem-klimatologam-...,indicator.ru,https://indicator.ru/article/2017/01/04/chem-k...,2017_01_04,"Плюсы, минусы, коралловые рифы",Чем климатологам запомнился 2016 год,Дарья Сапрыкина,Науки о Земле,,Статьи
0,/pages_indicator/2017_05_22_patent-na-pianolu_,indicator.ru,https://indicator.ru/article/2017/05/22/patent...,2017_05_22,История науки: окончательный рассказ о механич...,117 лет назад был выдан патент на пианолу,,Технические науки,,Статьи
0,/pages_indicator/2016_09_30_boeing_,indicator.ru,https://indicator.ru/article/2016/09/30/boeing/,2016_09_30,История науки: мечта в металлическом корпусе,135 лет назад родился основатель фирмы «Боинг»,Алёна Манузина,Технические науки,,Статьи
0,/pages_indicator/2017_08_09_bibliometriya-ot-m...,indicator.ru,https://indicator.ru/article/2017/08/09/biblio...,2017_08_09,Наука подобрать: помогает ли наукометрия прини...,"Астроном Майкл Курц о том, как использовать на...",,Астрономия,,Статьи
0,/pages_indicator/2017_03_18_tri-v-odnom-lekars...,indicator.ru,https://indicator.ru/article/2017/03/18/tri-v-...,2017_03_18,"Три в одном: лекарства, ДНК и магниторецепция....",Трансляция биологического лектория Indicator.R...,,Биология,,Статьи


Сохраняем полученный датафрейм в отдельный файл формата csv (comma separated values)

In [47]:
ready_df.to_csv('indicator.csv', sep=',')